In [ ]:
%pip install -r ../Docs/requirements.txt --quiet

In [1]:
try:
    import os
    import mmap
    import lzma
    import string
    import random
    import pickle
    import pyperclip
    import torch
    import torch.nn as nn
    from torch.nn import functional as F
    from tqdm import tqdm
except Exception as e:
    print(f"Error : {e}")

In [ ]:
print(f"PyTorch version: {torch.__version__}")

def get_ipython():
    try:
        # Attempt to import IPython's get_ipython function
        from IPython import get_ipython as _get_ipython
        # Call the actual get_ipython function and return its result
        return _get_ipython()
    except ImportError:
        # If IPython is not available, return None
        return None

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is Apple MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is Apple MPS available? {torch.backends.mps.is_available()}")

if "google.colab" in str(get_ipython()):
    if torch.cuda.is_available():
        device = "cuda"
    else:
        print("GPU not available in Colab, consider enabling a GPU runtime") 

else:
    if torch.backends.mps.is_available():
        device = 'mps'
    elif torch.cuda.is_available():
        device = 'cuda'
    else:
        device = 'cpu'

print(f"Using device: {device}")

In [ ]:
div_len = 58
batch_size = 64
block_size = 128
max_iters = 1000
eval_iters = 100
learning_rate = 3e-4
n_embd = 384
n_layer = 8
n_head = 8
dropout = 0.2

In [ ]:
def get_directory():
    choice = input("Have you copied the path of the destination file? (Yes or No): ").lower()

    if choice in ['yes', 'y']:
        destination_path = pyperclip.paste()
        return destination_path.strip()
    else:
        print("To copy a file path, right-click the file, select 'Copy as path', then paste it here.")

downloads_dir = get_directory()

print("Destination path:", downloads_dir)

In [ ]:
def xz_files_in_dir(directory):
    files = []
    for filename in os.listdir(directory):
        if filename.endswith(".xz") and os.path.isfile(os.path.join(directory, filename)):
            files.append(filename)
    return files

owt_dir = '../Datasets/OpenWebText'
if not os.path.exists(owt_dir):
    os.makedirs(owt_dir)

train_filepath = f'{owt_dir}/train.txt'
val_filepath = f'{owt_dir}/val.txt'
vocab_filepath = f'{owt_dir}/vocab.txt'

files = xz_files_in_dir(downloads_dir)
total_files = len(files)
print(f"Files in directory: {total_files}")

split_index = int(total_files * 0.9)
files_train = files[:split_index]
files_val = files[split_index:]

vocab = set()

with open(train_filepath, "w", encoding="utf-8") as outfile:
    for filename in tqdm(files_train, total=len(files_train)):
        file_path = os.path.join(downloads_dir, filename)
        with lzma.open(file_path, "rt", encoding="utf-8") as infile:
            text = infile.read()
            outfile.write(text)
            characters = set(text)
            vocab.update(characters)

with open(val_filepath, "w", encoding="utf-8") as outfile:
    for filename in tqdm(files_val, total=len(files_val)):
        file_path = os.path.join(downloads_dir, filename)
        with lzma.open(file_path, "rt", encoding="utf-8") as infile:
            text = infile.read()
            outfile.write(text)
            characters = set(text)
            vocab.update(characters)

with open(vocab_filepath, "w", encoding="utf-8") as vfile:
    for char in vocab:
        vfile.write(char + '\n')

In [ ]:
def clean_text(text):
    printable_chars = set(string.printable)
    cleaned_text = ''.join(filter(lambda x: x in printable_chars, text))
    return cleaned_text

cleaned_text = clean_text(text)
chars = sorted(list(set(cleaned_text)))

print(len(cleaned_text))
print(chars)
print(len(chars))
vocab_size = len(chars)

In [ ]:
chars = ""
with open(vocab_filepath, 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

print(len(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

In [ ]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

def get_random_chunk(split):
    filename = train_filepath if split == 'train' else val_filepath
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size * batch_size)
            mm.seek(start_pos)
            block = mm.read(block_size * batch_size - 1)
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
    return data

In [ ]:
def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

Reference: (https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module)

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class PersonalLLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat([index, index_next], dim=1)
        return index
    
model = PersonalLLM(vocab_size)
m = model.to(device)

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() 
    return out

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

print("Lower is better")
print('-'*div_len)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step: {iter}, Training loss: {losses['train']:.3f}, Validation loss: {losses['val']:.3f}")
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print('-'*div_len)
print(f'Final loss: {loss.item():.3f}')

In [ ]:
model_extn = '.pkl'
model_title = 'PersonalLLM'
model_name = f'{model_title}_{n_layer}x{n_head}_EMBD{n_embd}_{batch_size}x{block_size}{model_extn}'
model_dir = '../Models/'
model_pathname = os.path.join(model_dir, model_name)

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

with open(model_pathname, 'wb') as file:
    pickle.dump(model, file)

print("-"*div_len)
print(f"Model saved successfully as '{model_name}' file.")
print("-"*div_len)